In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import zipfile

zip_path = "/content/drive/MyDrive/processed_data.zip"

In [3]:
with zipfile.ZipFile(zip_path, "r") as zip_ref:
  zip_ref.extractall("/content")

print("Dataset extracted successfully...")

Dataset extracted successfully...


In [4]:
import numpy as np

X_train = np.load("/content/processed_data/X_for-norm_training_scaled.npy")
X_val = np.load("/content/processed_data/X_for-norm_validation_scaled.npy")
X_test = np.load("/content/processed_data/X_for-norm_testing_scaled.npy")

y_train = np.load("/content/processed_data/y_for-norm_training.npy")
y_val = np.load("/content/processed_data/y_for-norm_validation.npy")
y_test = np.load("/content/processed_data/y_for-norm_testing.npy")

print("Data Loaded Successfully!")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

Data Loaded Successfully!
X_train shape: (53868, 40, 157)
y_train shape: (53868,)


In [5]:
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print("New X_train shape:", X_train.shape)

New X_train shape: (53868, 40, 157, 1)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM, Reshape

In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(40, 157, 1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Reshape((-1, 64)),
    LSTM(64, return_sequences=False),

    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 38, 155, 32)         │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 38, 155, 32)         │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 19, 77, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 17, 75, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 17, 75, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 37, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 296, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,449 (220.50 KB)

 Trainable params: 56,257 (219.75 KB)

 Non-trainable params: 192 (768.00 B)

In [10]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

Epoch 1/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 48s 24ms/step - accuracy: 0.8968 - loss: 0.2537 - val_accuracy: 0.9614 - val_loss: 0.1047
Epoch 2/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.9669 - loss: 0.0949 - val_accuracy: 0.9790 - val_loss: 0.0598
Epoch 3/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.9767 - loss: 0.0678 - val_accuracy: 0.9741 - val_loss: 0.0859
Epoch 4/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 40s 24ms/step - accuracy: 0.9846 - loss: 0.0473 - val_accuracy: 0.9816 - val_loss: 0.0564
Epoch 5/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 43s 25ms/step - accuracy: 0.9846 - loss: 0.0458 - val_accuracy: 0.9856 - val_loss: 0.0453
Epoch 6/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 80s 24ms/step - accuracy: 0.9888 - loss: 0.0348 - val_accuracy: 0.9795 - val_loss: 0.0609
Epoch 7/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.9888 - loss: 0.0318 - val_accuracy: 0.9887 - val_loss: 0.0368
Epoch 8/20
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.9900 -

In [11]:
model.save("/content/deepfake_voice_model.h5")
print("Model saved successfully...")

Model saved successfully...
